#Рекуррентные нейросети для классификации текстов
В этом уроке мы рассмотрим различные компоненты, которые помогут нам создать полноценный классификатор последовательностей (текстов) в будущем.

В том числе рассмотрим реализацию двунаправленных LSTM сетей.

### Используем TensorFlow 2.0

На момент подготовки этих материалов в Google Colab по умолчанию используется версия TensorFlow 1.X

Переключаемся на версию 2.0 (работает только в Colab)

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


### Загрузка библиотек
TensorFlow должен иметь как минимум версию 2.0

In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.0.0


### Входной тензор
Как и раньше, создадим тензор входных тестовых цепочек. 

Однако, теперь, давайте работать не с произвольными векторами, а со словами.
Зачастую нам дан (или мы можем создать) словарь из всех используемых слов. Тогда каждое слово можно кодировать индексом в этом словаре (просто целым числом). Предобработку текста (нормализацию слов итд) оставим за рамками.

В таком случае входной тензор будет просто двумерным целочисленным тензором (каждый его элемент - индекс слова в словаре).

Размерности тензора `x`: (батч, длина цепочки)

Но так как в рекуррентную нейронную сеть нам всё равно надо передавать векторные представления слов, нам далее потребуется дополнительный шаг -- получение эмбеддингов. Один из самых простых способов сделать это -- создать обучаемый `Embedding` слой, который будет просто содержать матрицу из эмбеддингов для каждого слова из словаря. Для создания такого слоя нам понядобятся параметры `EMB_SIZE` (размерность эмбеддинга) и `VOCAB_SIZE` (количество слов в словаре).

Мы всё еще не решаем никакую конкретную задачу, а просто демонстрируем то, как можно создать и использовать различные рекуррентные сети на абстрактных тестовых данных.


In [ ]:
BATCH_SIZE = 2
SEQ_LEN = 100 # Длина последовательности
EMB_SIZE = 16 # Размер векторного представления (эмбеддинга)
VOCAB_SIZE = 10000 # Количество слов в словаре

x = np.random.randint(VOCAB_SIZE, size=(BATCH_SIZE, SEQ_LEN))
print(x.shape)

(2, 100)


### Обучаемый Embedding слой
Создадим Embedding слой, который отображает словарный индекс слова в соответствующий эмбеддинг (вектор).

По сути, это просто обучаемая матрица размера `(VOCAB_SIZE, EMB_SIZE)`, которая по индексу слова `i` выдает i-ый столбец этой матрицы.

Такой слой можно использовать как простую альтернативу Word2Vec и других векторных представлений слов.

Если мы применим Embedding слой для нашего входного тензора, получим уже эмбеддинги (векторы) для каждого слова в цепочке. То есть выходной тензор будет иметь размер `(BATCH_SIZE, SEQ_LEN, EMB_SIZE)`

In [ ]:
embedding_layer = tf.keras.layers.Embedding(VOCAB_SIZE, EMB_SIZE)

y = embedding_layer(x)
print(y.shape)

(2, 100, 16)


### Пример простой рекуррентной нейронной сети для бинарной классификации текстов
Теперь у нас есть почти всё, что нужно для создания классификатора текстов.

Один из самых простых классификаторов можно создать из следующих трёх блоков: Embedding слой, рекуррентный слой, который возвращает лишь один вектор для каждой цепочки (`return_sequences=False`), Полносвязный слой, отображающий полученный промежуточный вектор в классификационный ответ. Например, если у нас бинарный классификация (на два класса), можно в конце использовать один нейрон с функцией активации sigmoid (как в обычных нейросетевых классификаторах)

Создадим соответствующую Sequential модель, вызовем инференс и посмотрим на размер ответа. 

У нас на входе был батч из цепочек. Каждую цепочку мы хотим проклассифицировать. Ответ классификатора для цепочки это одно число (вероятность после сигмоида). Тогда размер выходного тензора должен получиться `(BATCH_SIZE, 1)`

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.LSTM(16, return_sequences=False),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

y = model(x)
print(y.shape)

(2, 1)


### Регуляризация
Рассмотрим различные модификации и улучшения текстового классификатора.

Рекуррентные сети часто страдают от проблемы переобучения, поэтому к ним необходимо применять один из стандартных способов борьбы с переобучением -- dropout (случайное отключение части нейронов во время обучения).

В LSTM слое можно устанавливать различные показатели дропаута на разные участки слоя с помощью параметров `dropout` и `recurrent_dropout`

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

y = model(x)
print(y.shape)

(2, 1)


### Более глубокая рекуррентная сеть
Можно ставить несколько LSTM слоёв, делая сеть глубже.

В таком случае, все кроме последнего LSTM слоя должны иметь параметр `return_sequences=True`, иначе последующий слой не сможет обработать выход предыдущего (он ожидает последовательность, а ему дают лишь один объект)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.LSTM(16, return_sequences=True, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.5, recurrent_dropout=0.5),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

y = model(x)
print(y.shape)

(2, 1)


### Двунаправленные рекуррентные сети
До этого моменты мы использовали однонаправленные рекуррентные сети. В случае классификации текстов могут помочь улучшить результат двунаправленные сети (которые считывают входную цепочку в двух противоположных направлениях).

В Keras двунаправленность можно добавить просто обернув LSTM слой в `tf.keras.layers.Bidirectional()`.

Сделаем это для всех наших LSTM слоёв.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 16),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(16, return_sequences=True, dropout=0.5, recurrent_dropout=0.5)),
    tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

y = model(x)
print(y.shape)

(2, 1)


###Заключение
Рассмотренных элементов достаточно для построения и обучения полноценного классификатора текстов, чем мы и займёмся в следующем уроке.